# なろう小説APIによるデータの取得

In [1]:
import requests
import pandas as pd
import json
import time as tm
import datetime
import gzip
from tqdm.notebook import tqdm
import xlsxwriter
import sqlite3

In [2]:
#『なろう小説API』を用いて、なろうの『全作品情報データを一括取得する』Pythonスクリプト
#2020-04-26更新

#リクエストの秒数間隔(1以上を推奨)
interval = 2
### なろう小説API・なろう１８禁小説API を設定 ####
is_narou = True
now_day = datetime.datetime.now()
now_day = now_day.strftime("%Y_%m_%d")
if is_narou:
    filename = 'Narou_All_OUTPUT_%s.xlsx'%now_day
    sql_filename = 'Narou_All_OUTPUT_%s.sqlite3'%now_day
    api_url="https://api.syosetu.com/novelapi/api/"    
else:
    filename ='Narou_18_ALL_OUTPUT_%s.xlsx'%now_day
    sql_filename = 'Narou_18_ALL_OUTPUT_%s.sqlite3'%now_day
    api_url="https://api.syosetu.com/novel18api/api/"
# データをSqlite3形式でも保存する
is_save_sqlite = False
#####　以上設定、以下関数　##############
    
#全作品情報の取得
def get_all_novel_info():
       
    df = pd.DataFrame()
    
    payload = {'out': 'json','gzip':5,'of':'n','lim':1}
    res = requests.get(api_url, params=payload).content
    r =  gzip.decompress(res).decode("utf-8") 
    allcount = json.loads(r)[0]["allcount"]
    print('対象作品数  ',allcount);
    
    all_queue_cnt = (allcount // 500) + 10
    
    #現在時刻を取得
    nowtime = datetime.datetime.now().timestamp()
    lastup = int(nowtime)
                     
    for i in tqdm(range(all_queue_cnt)):
        payload = {'out': 'json','gzip':5,'opt':'weekly','lim':500,'lastup':"1073779200-"+str(lastup)}
        
        # なろうAPIにリクエスト
        cnt=0
        while cnt < 5:
            try:
                res = requests.get(api_url, params=payload, timeout=30).content
                break
            except:
                print("Connection Error")
                cnt = cnt + 1
                tm.sleep(120) #接続エラーの場合、120秒後に再リクエストする
            
        r =  gzip.decompress(res).decode("utf-8")   
    
        # pandasのデータフレームに追加する処理
        df_temp = pd.read_json(r)
        df_temp = df_temp.drop(0)
        df = pd.concat([df, df_temp])
        
        last_general_lastup = df.iloc[-1]["general_lastup"]
        lastup = datetime.datetime.strptime(last_general_lastup, "%Y-%m-%d %H:%M:%S").timestamp()
        lastup = int(lastup)
        #取得間隔を空ける
        tm.sleep(interval)


In [3]:
df = get_all_novel_info()

対象作品数   850280


  0%|          | 0/1710 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# 保存
df.to_pickle("../data/narou.pkl")

In [4]:
# 読み込み
df = pd.read_pickle("../data/narou.pkl")
df

,title,ncode,userid,writer,story,biggenre,genre,gensaku,keyword,general_firstup,...,impression_cnt,review_cnt,all_point,all_hyoka_cnt,sasie_cnt,kaiwaritu,novelupdated_at,updated_at,weekly_unique,log_global_point
0,シリアナの女王と破滅のアジフライ,N8985GS,792128,λμ,今ではない、いつかどこかの霧の街。\n金髪に黒い瞳の美少年、デリは夕餉の支度をすませ祖父の帰...,2,202,NaN,R15 残酷な描写あり 伝奇 サイコホラー 異能力バトル エロ グロ ナンセンス おねショタ...,2021-01-19 20:37:09,...,0,0,10,1,0,44,2021-02-06 00:18:43,2021-02-06 00:22:04,112,2.564949
1,もしも浪漫竒譚～あやかしと生きる民らの黄昏,N2308GO,1911237,やばいやつ,舞台は、「もしも」の明治時代。西欧化と戦争の混乱、アンニュイな空気を背景に、「神和警邏団（か...,3,303,NaN,R15 残酷な描写あり ヒストリカル 時代小説 異能力バトル シリアス 男主人公 和風 明治...,2020-10-17 02:58:14,...,0,0,0,0,17,47,2021-02-06 00:18:36,2021-02-06 00:22:04,0,1.098612
2,5％の冷やした砂糖水,N6532GT,2106731,煙 うみ,研修医１年目。喫煙者でピアスだらけの私は職場に馴染めず、ワンナイトした元親友は連絡をくれない...,3,302,NaN,オフィスラブ 日常 私小説 医療 現代,2021-02-02 22:07:00,...,0,0,0,0,0,23,2021-02-06 00:18:33,2021-02-06 00:22:04,0,1.098612
3,喫茶猫屋【連載版】,N2872GN,1440177,羽生田 鷹,喫茶“猫屋”にて繰り広げられる\n日常と非日常の物語\n\nもし、猫になれるとしたらなりますか？,4,403,NaN,日常 男主人公 人外 現代 王 動物 特殊能力,2020-09-27 11:00:00,...,0,0,0,0,0,30,2021-02-06 00:18:08,2021-02-06 00:22:04,0,0.000000
4,閃光の魔術師～六属性で攻撃魔法がない光魔法しか使えない？でも俺には頼れる仲間がいる！絶対にこ...,N4462GP,2049322,クロア,現代から異世界に転生した主人公。\n\n✱女神は主人公の願いを叶える為に呼び寄せた▲\n\n...,2,201,NaN,R15 異世界転生 日常 青春 ミステリー 冒険 ラブコメ 男主人公 学園 ハーレム シリア...,2020-11-12 09:42:07,...,4,5,186,20,1,26,2021-02-06 00:17:52,2021-02-06 00:20:05,1790,6.037871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788725,軍（いくさ）神の剣,N0080A,23,earl gray,軍神チュールの苦悩。ヒルドを奪われた悲しみから、チュールは・・・・・・。,98,9801,NaN,ファンタジー 北欧神話 恋愛,2004-05-20 17:11:21,...,1,0,38,5,0,44,2009-09-30 10:02:26,2021-01-31 04:04:03,0,3.970292
788726,灰色の涙,N0084A,29,綾月,ひょんな事から、大学生の吉田啓介は記憶喪失に陥った少女を遭遇する…そして啓介はその彼女の思い...,98,9801,NaN,恋愛 記憶喪失 大学生,2004-05-23 00:38:17,...,0,0,10,3,0,35,2009-09-30 10:02:27,2021-02-03 04:04:03,0,2.944439
788727,North Land Storen,N0081A,23,earl gray,不老不死の（あくどい）天才錬金術師ヘルギと弟子でスケベなジュリアーノの、まともじゃない冒険記...,98,9801,NaN,ファンタジー 錬金術,2004-05-20 17:26:37,...,1,0,18,3,0,54,2009-09-30 10:02:26,2021-01-30 04:04:03,0,3.295837
788728,傲慢なる少年の傀儡,N0052A,44238,水無月冬弥,ゲームセンターを牛耳り、警察さえ手が出せない少年とその護衛。＜悪魔＞の騎士字伏戒那は、秘策と...,98,9801,NaN,現代ファンタジー,2004-05-02 01:40:21,...,9,0,96,12,0,35,2009-09-30 10:02:20,2021-02-02 04:04:04,0,4.844187


In [5]:
df[["title","story"]].head(5)

,title,story
0,シリアナの女王と破滅のアジフライ,今ではない、いつかどこかの霧の街。\n金髪に黒い瞳の美少年、デリは夕餉の支度をすませ祖父の帰...
1,もしも浪漫竒譚～あやかしと生きる民らの黄昏,舞台は、「もしも」の明治時代。西欧化と戦争の混乱、アンニュイな空気を背景に、「神和警邏団（か...
2,5％の冷やした砂糖水,研修医１年目。喫煙者でピアスだらけの私は職場に馴染めず、ワンナイトした元親友は連絡をくれない...
3,喫茶猫屋【連載版】,喫茶“猫屋”にて繰り広げられる\n日常と非日常の物語\n\nもし、猫になれるとしたらなりますか？
4,閃光の魔術師～六属性で攻撃魔法がない光魔法しか使えない？でも俺には頼れる仲間がいる！絶対にこ...,現代から異世界に転生した主人公。\n\n✱女神は主人公の願いを叶える為に呼び寄せた▲\n\n...


# タイトルの学習

In [6]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import re


import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

from sklearn.model_selection import train_test_split,KFold
import random
import logging


# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [7]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 300

## データの読み込み

In [8]:
narou = pd.read_pickle("../data/narou.pkl")
narou = narou.dropna(subset=["title","story"])
narou["title"] = narou["title"].astype(str)
narou["story"] = narou["story"].astype(str)

In [9]:
narou = narou[narou["novelupdated_at"].dt.year==2020].copy()

In [10]:
narou.head(5)

,title,ncode,userid,writer,story,biggenre,genre,gensaku,keyword,general_firstup,general_lastup,novel_type,end,general_all_no,length,time,isstop,isr15,isbl,isgl,iszankoku,istensei,istenni,pc_or_k,global_point,daily_point,weekly_point,monthly_point,quarter_point,yearly_point,fav_novel_cnt,impression_cnt,review_cnt,all_point,all_hyoka_cnt,sasie_cnt,kaiwaritu,novelupdated_at,updated_at,weekly_unique,log_global_point
29025,昨日の私を超えてゆけ,N3876FY,496462,夏野 夜子,新年なので思い立って、毎月何かしら新しいことをすることにした。\nそして得たものを記録しようと思った。\nこれは挫折と戦う夢と希望とやる気と運命と計画力とかの日々の記録である。,99,9903,NaN,記録 ゆるチャレンジ ゆるエッセイ 映画,2020-01-01 23:08:58,2020-12-31 23:58:30,1,0,367,521997,1044,0,0,0,0,0,0,0,2,878,0,12,18,248,570,129,54,0,620,67,76,1,2020-12-31 23:58:30,2021-02-04 05:17:29,158,6.778785
29028,輪廻,N9233GR,1577885,国岡徹,輪廻は回る。\n君の歯茎や、目の前のコーヒーであってもだ。,3,301,NaN,私小説 純文学 コーヒー 喫茶店 哲学 輪廻 癒着 帰結,2020-12-31 23:55:08,2020-12-31 23:55:08,2,0,1,1486,3,0,0,0,0,0,0,0,2,2,0,0,0,2,2,1,0,0,0,0,0,38,2020-12-31 23:55:08,2021-02-01 06:17:59,0,1.098612
29029,DREAM DIVER,N9971DX,1042426,坂田クロキ,\n時はC.E 2139年………人類は新たな病に悩まされていた。\n\n“夢迷い病”――通称『Dream Lost Syndrome(ドリーム・ロスト・シンドローム)』と呼ばれる奇病。これに罹ると、夢を夢と認識出来なくなり、眠りについたまま目覚めなくなる。そう、まさに夢の世界で迷い子になってしまうのだ……。\n\nその病気から回復する手段はただ一つ、『Dream Diver（ドリーム・ダイバー）』ーー“WD”とも呼ばれるーー彼らによる治療を受けること。彼らは夢迷い病の患者の夢に、まるで潜るようにダイブする。夢と現実の狭間で、彼らは何を見、また何を思うのかーー……。\n\n現実（リアル）...,4,403,NaN,近未来 人工知能 電脳世界 サイバーパンク SF 攻殻機動隊 INCEPTION パプリカ,2017-04-18 22:26:56,2020-12-31 23:55:04,1,1,25,101775,204,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,62,2020-12-31 23:55:04,2021-01-01 04:55:06,0,0.000000
29030,ヴァンパイアZERO,N2842EA,683700,K糸,祭苑市ーーその街では二つの事件が起きていた。\n赤月の天園__ルベル・カエルムと呼ばれるドラッグの蔓延。もう一つは吸血鬼と呼ばれる連続殺人事件。\n祭苑市に住む学生・本條暁人は吸血鬼殺人事件の犯人を追うために本物の吸血鬼と接触し自分も吸血鬼へと新生する。そして狩人をも巻き込み事件は吸血鬼達の闘争へと色を変え、終息へと向かう。\n滅びされ、夜の魔人共よ。消え失せろ、泡沫の幻想よ。これより始まるのは吸血鬼の滅亡劇。,2,202,NaN,R15 残酷な描写あり 異能力バトル 吸血鬼 ダーク シリアス ホラー？ マルチエンド(予定) ヴァンパイア,2017-05-31 06:49:07,2020-12-31 23:54:20,1,1,10,50546,102,0,1,0,0,1,0,0,2,8,0,0,0,0,0,1,0,0,6,1,0,51,2020-12-31 23:54:20,2021-01-01 04:56:30,0,2.197225
29031,戦艦に魅せられて,N9232GR,1128087,さよなら2020,何か書きたかった\n戦艦って最高だろ？\nわかりやすい強さっていいよね？\n実際に見てみたかったてのと、こんなん無かったほうがいいって思いが入れ混じってる\nでも見てるだけなら兵器って心踊るよね\n歴史上の出来事も。\nまぁ生きてる間に戦争に兵士として行くことも無いといいけどね,3,303,NaN,史実,2020-12-31 23:54:11,2020-12-31 23:54:11,2,0,1,697,2,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,2020-12-31 23:54:11,2021-01-05 08:35:09,0,0.000000


## データセットの定義

In [11]:
tokenizer = T5Tokenizer.from_pretrained("sonoisa/t5-base-japanese")

In [12]:
pad = tokenizer.get_vocab()['<pad>']

In [13]:
# 上記のinput_dataの作成方法をもとにdataset化
class NarouDataset(torch.utils.data.Dataset):
    def __init__(self, input_texts,input_max_len, target_texts = None,target_max_len=None):
        self.train_flg = 0 if target_texts is None else 1
        output_dict = {"input_ids":[],"input_masks":[],"target_ids":[],"target_masks":[]}
        temp_dicts = tokenizer.batch_encode_plus(
                    input_texts, max_length=input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )
        
        self.input_ids = temp_dicts["input_ids"]
        self.input_masks = temp_dicts["attention_mask"]
        if self.train_flg:
            temp_dicts = tokenizer.batch_encode_plus(
                        target_texts, max_length=input_max_len, truncation=True, 
                        padding="max_length", return_tensors="pt"
                    )

            self.target_ids = temp_dicts["input_ids"]
            self.target_masks = temp_dicts["attention_mask"]
        

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.train_flg:
                return (self.input_ids[idx].squeeze(), self.input_masks[idx].squeeze(),
                        self.target_ids[idx].squeeze(), self.target_masks[idx].squeeze() )
        else:
                return self.input_ids[idx].squeeze(), self.input_masks[idx].squeeze()

In [14]:
X_train, X_test, y_train, y_test = train_test_split(narou["story"],narou["title"],test_size=0.1,random_state = 1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state = 1)

In [15]:
# datasetとdataloaderの作成
batch_size=2
train_ds = NarouDataset(input_texts=X_train,input_max_len=256, target_texts = y_train,target_max_len=100)
train_dl = torch.utils.data.DataLoader(
    train_ds, batch_size=batch_size, shuffle=True)  
valid_ds = NarouDataset(input_texts=X_valid,input_max_len=256, target_texts = y_valid,target_max_len=100)
valid_dl = torch.utils.data.DataLoader(
    valid_ds, batch_size=batch_size, shuffle=False)

## モデルの定義

In [16]:
model = T5ForConditionalGeneration.from_pretrained("sonoisa/t5-base-japanese")

In [17]:
cuda = torch.cuda.is_available()
#cuda = False
# 高速化
torch.backends.cudnn.benchmark = True
if cuda:
    model.cuda()
# optimizerの設定
optimizer = optim.Adam(model.parameters(),lr = 5e-5 ,betas=(0.9, 0.999))

# 損失関数の設定
criterion = nn.CrossEntropyLoss()

## 学習

In [18]:
## 学習します。
epochs = 5
best_loss = 9999999
for epoch in range(epochs):
    total_loss = 0
    model.train()
  

    print("epoch {} start!".format(epoch+1))
    logging.info("epoch {} start!".format(epoch+1))
    # train
    for iter_num, (input_ids, input_masks, target_ids, target_masks) in tqdm(enumerate(train_dl),total = len(train_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, input_masks, target_ids, target_masks =\
            input_ids.cuda(), input_masks.cuda(), target_ids.cuda(), target_masks.cuda()
        if len(input_ids)>1:
            # forward(順伝搬)
            outputs = model(
                input_ids=input_ids,
                attention_mask=input_masks,
                labels=target_ids)
            loss = outputs.loss

            # backward(逆伝搬)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            #50 iterごとにlossとaccuracyを表示
            if (iter_num+1) % 100 == 0:
                size = batch_size*(iter_num+1)
                print("{} iter loss:{:.4f}".format(iter_num+1,total_loss/(iter_num+1)))
                logging.info("{} iter loss:{:.4f}".format(iter_num+1,total_loss/(iter_num+1)))

    total_loss /= len(train_dl)

    # validation
    val_total_loss = 0
    model.eval()
    for iter_num, (input_ids, input_masks, target_ids, target_masks) in tqdm(enumerate(valid_dl),total = len(valid_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, input_masks, target_ids, target_masks =\
            input_ids.cuda(), input_masks.cuda(), target_ids.cuda(), target_masks.cuda()
        with torch.no_grad():
            # forward(順伝搬)
            outputs = model(
                input_ids=input_ids,
                attention_mask=input_masks,
                labels=target_ids)
            loss = outputs.loss
            val_total_loss += loss.item()
            


    val_total_loss /= len(valid_dl)

    print("epoch{} total loss:{:.4f}, val_total loss:{:.4f}".format(epoch+1,total_loss,val_total_loss))
    logging.info("epoch{} total loss:{:.4f}, val_total loss:{:.4f}".format(epoch+1,total_loss,val_total_loss))
    if best_loss > val_total_loss:
        print(f"epoch{epoch+1} model saved")
        logging.info(f"epoch{epoch+1} saved")
        torch.save(model.state_dict(), f'../model/T5model{epoch}.hdf5')
        best_loss = val_total_loss


epoch 1 start!


  0%|          | 0/52487 [00:00<?, ?it/s]

100 iter loss:53.2120
200 iter loss:47.2292
300 iter loss:38.3150
400 iter loss:29.1576
500 iter loss:23.4362
600 iter loss:19.6039
700 iter loss:16.8561
800 iter loss:14.7987
900 iter loss:13.1956
1000 iter loss:11.9119
1100 iter loss:10.8612
1200 iter loss:9.9855
1300 iter loss:9.2439
1400 iter loss:8.6087
1500 iter loss:8.0563


KeyboardInterrupt: 

# タイトルの予測

## X_testの予測

In [19]:
best_epoch = 2
model_path = f'../model/T5model{best_epoch}.hdf5'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [20]:
# datasetとdataloaderの作成
batch_size=2
test_ds = NarouDataset(input_texts=X_test,input_max_len=256)
test_dl = torch.utils.data.DataLoader(
    test_ds, batch_size=batch_size, shuffle=False)  

In [21]:
def pred_text(X_test):
    # datasetとdataloaderの作成
    batch_size=1
    test_ds = NarouDataset(input_texts=X_test,input_max_len=256)
    test_dl = torch.utils.data.DataLoader(
        test_ds, batch_size=batch_size, shuffle=False)  
    pred_texts = []
    model.eval()
    for iter_num, (input_ids, input_masks) in tqdm(enumerate(test_dl),total = len(test_dl)):
        optimizer.zero_grad()
        if cuda:
            input_ids, input_masks =\
            input_ids.cuda(), input_masks.cuda()
        with torch.no_grad():
            # forward(順伝搬)
            outputs = model.generate(input_ids)
            for pred_ids in outputs:
                if (pred_ids==1).nonzero().shape[0]:
                    pred_texts.append(tokenizer.decode(pred_ids[1:(pred_ids==1).nonzero()[0,0]]))
                else:
                    pred_texts.append(tokenizer.decode(pred_ids[1:]))
    return pred_texts

In [22]:
preds = pred_text(X_test)

  0%|          | 0/14580 [00:00<?, ?it/s]

/home/shingo/anaconda3/envs/py37/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [23]:
test_df = pd.DataFrame({
    "title":y_test,    
    "pred_title":preds,
    "story":X_test
})

In [24]:
test_df["len"] = test_df["story"].map(len)

In [25]:
test_df[(test_df["len"]>100) & (~test_df["pred_title"].str.contains("<unk>"))]

,title,pred_title,story,len
33452,冤罪探偵,ヘッポコ刑事とポンコツ探偵の事件簿,うっかり殺人を犯してしまった島村幸次郎。\nだが捜査に当たったヘッポコ刑事とポンコツ探偵は大ハズレの推理と共に別の人間を逮捕し、結果捕まらずに済んだ島村は呑気にハンバーガーを齧っていた。\n\nしかし安堵したのもつかの間、翌日彼の前にはまたもヘッポコ・ポンコツコンビの姿が。\n\nまさか真相がバレてしまったのか!? と島村は身構えるも、どうやら彼らは例の事件と同時に発生した凶悪事件の真犯人として島村を連行しようとしているようであった。\n\n無論それは冤罪であるのだが、無実を証明するには逮捕を免れた事件について触れなければならない。\n\n「三日だ！ 三日で本当の真犯人、俺が見つけてや...,504
112616,幹に取り込まれた枝,森のログハウス,両親に連れられて、森のログハウスに来た小学生の兄弟。\nログハウスの材料である、この森の木は、木の節が異常に多くなるという。\n森の奥に足を踏み入れたその兄弟は、この森の木に節が多くなる理由を知ることになる。\n,104
241991,Continue To Question,死なせてくれませんか?,順風満帆だったはずの人生に初めて挫折を味わう主人公。何故こんなにも悩まなければならない？何故自分は生きているの？何故死なせてくれないの？何故生まれてきてしまったの？正解の無い問いに主人公はどう向き合うのか、どんな答えを見つけるのか,115
164054,美人双子に愛され過ぎて、気づいたら監禁犯になっていた,美人双子に監禁シリーズ 第二部,【美人双子に監禁シリーズ、第二部開幕――】\n「ん……グーテンモルゲン？」\n\n どこにでもいる大学生の不壇通哲也（ふだんどおり てつや）が目を覚ますと、腹の上に金髪碧眼の美幼女が乗っていた。\n 驚いてリビングに駆け込む哲也を待っていたのは、神妙な面持ちで鎖が千切れた足枷を手にする涼天咲月（りょうてん さつき）と怪しいデータベースを調査（ハッキング）する涼天咲夜（さくや）の双子の姉妹。\n まったく面識のない金髪碧眼幼女の監禁に手を貸すハメになった哲也は、夏休みの間、奇妙な監禁生活を強いられることに。\n\n「ここは世界で一番の【愛の檻】――俺達が、この子の『家族』で【檻】になろ...,653
252424,英雄たちの歓喜,霊感少年と悪鬼の宿命,ある日。通り魔に襲われ、生死をさ迷った少年、長谷川栄一は、ふと目を覚ますと悪霊が見えるようになってしまっていた。\n\n 霊感少年として高校生活を送ることになった栄一だが、気がつくと彼は沢山の霊能力者たちに囲まれることに。\n\n やがて彼は、町に怪異を運ぶ数々の悪鬼と戦う宿命に巻き込まれるのであった。,149
...,...,...,...,...
41958,監禁ゲームのヒロインはトゥルーエンドを目指さない,監禁エンドは嫌なので、ハーレムエンドを目指します!,いくら愛されていても監禁は嫌だ。まして死ぬだなんて。\n\n アメリは学園の王子様クラウスに出会った瞬間、前世の自分を思い出した。そして、ここがゲーム「エターナルキャッスル」の世界だということも。\n\n ヒロインである自分が、クラウスルート突入のフラグを立ててしまったと気づいたアメリは、トゥルーエンド回避のため、ハーレムエンドを目指すことにする。\n\n 何故ならこのゲームはすべての攻略対象でノーマルエンドが監禁エンドなのだ。しかもクラウスのトゥルーエンドでは殺されてしまう……。\n\n しかし、この世界のクラウスは「俺のことは好きにならないでほしい」と言う。他の攻略対象もゲームと...,452
41203,ペット化した女神とほのぼのスローライフを送ることになりました,わがまま毒舌ロリ女神がウザすぎて、俺の異世界ライフがとんでもないことになってるんですけど......!,わがまま毒舌ロリ女神がウザすぎて、俺の異世界ライフがとんでもないことになってるんですけど……！\n\n\nある日突如として天界に呼ばれた主人公コタツは、そこで出会った女神に自らが死亡したことを告げられる。その後訳の分からないまま転生する流れになったのだが、性格が残念すぎる女神に思いっきり雑な扱いを受け適当な処置で異世界に送りだされてしまった。\n思わず全てを諦めたコタツだったが、転生直後、なぜかすぐ隣に先ほどの女神がきょとんとした顔でつっ立っていた―。,224
123273,自称地味っ子公爵令嬢は婚約を破棄して欲しい？,地味な公爵令嬢は二度の恋に落ちました。,アメジシスト王国の王太子であるカウレスの婚約者の座は長い間空席だった。\nカウレスは、それはそれは麗しい美青年で婚約者が決まらないことが不思議でならないほどだ。\nそんな、麗しの王太子の婚約者に、何故か自称地味でメガネなソフィエラが選ばれてしまった。\nソフィエラは、麗しの王太子の側に居るのは相応しくないと我慢していたが、とうとう我慢の限界に達していた。\n意を決して、ソフィエラはカウレスに言った。\n「お願いですから、わたしとの婚約を破棄して下さい！！」\n意外にもカウレスはあっさりそれを受け入れた。しかし、これがソフィエラにとっての甘く苦しい地獄の始まりだったのだ。\nそして、カウ...,470
39091,アニメ業界改善に向けたアイデアBOX,アニメ業界のブラックな現状を変えたい。,閉鎖した「アニメ業界のブラックな現状を変えたい。」の反省を踏まえ、\nあくまでTwitter上の活動とは切り離した完全な個人活動とした上で、業界の部外者であるため知識不足の点や、至らぬ点はございますがアイデアを出していければと思います。\n\nアニメ業界の現場の労働条件が一日でも早く改善されることを望みます。,152


## 全ての年代のなろう小説で予測

In [26]:
narou_all = pd.read_pickle("../data/narou.pkl")
narou_all = narou_all.dropna(subset=["title","story"])
narou_all["title"] = narou_all["title"].astype(str)
narou_all["story"] = narou_all["story"].astype(str)

In [27]:
lists = ["蜘蛛ですが、なにか？","世界最速のレベルアップ　～無能スキル【ダンジョン内転移】が覚醒した結果、俺だけダンジョンのルールに縛られず最強になった～","Ｒｅ：ゼロから始める異世界生活","異世界居酒屋「のぶ」",
         "本好きの下剋上　～司書になるためには手段を選んでいられません～"]
narou_output = narou_all[narou_all["title"].isin(lists)].copy()
narou_output["pred_title"] = pred_text(narou_output["story"])
narou_output[["title","pred_title","story"]]

  0%|          | 0/5 [00:00<?, ?it/s]

,title,pred_title,story
319,蜘蛛ですが、なにか？,蜘蛛の魔物に転生した俺は蜘蛛の魔物に転生する,勇者と魔王が争い続ける世界。勇者と魔王の壮絶な魔法は、世界を超えてとある高校の教室で爆発してしまう。その爆発で死んでしまった生徒たちは、異世界で転生することになる。クラスの中でも最底辺に位置する主人公は、よりにもよって蜘蛛の魔物として生まれ変わってしまう。ただ、異常な程に強い精神力で現状を受け止め、割とあっさり順応してしまう。これは蜘蛛の魔物になってしまった主人公が、なんやかんやサバイバルして生きていく物語である。\nなんか書籍発売してるらしいですよ。
7501,Ｒｅ：ゼロから始める異世界生活,異世界召喚されたひきこもりの俺は、何度死んでも巻き戻ります,突如、コンビニ帰りに異世界へ召喚されたひきこもり学生の菜月昴。知識も技術も武力もコミュ能力もない、ないない尽くしの凡人が、チートボーナスを与えられることもなく放り込まれた異世界で必死こいて生き抜く。彼に与えられたたった一個の祝福は、『死んだら巻き戻ります』という痛みを伴う『死に戻り』のみ！ 頼れるもののいない異世界で、いったい彼は何度死に、なにを掴み取るのか。 ※血も死体も出る予定ですが、そんな派手なことにはなりません。\n\n※当作品は2014年1月24日、ＭＦ文庫Ｊ様の方から同タイトルで書籍化させていただいております。皆様の応援のおかげです、ありがとうございます。また、ＷＥＢ版の...
7636,世界最速のレベルアップ ～無能スキル【ダンジョン内転移】が覚醒した結果、俺だけダンジョンのルールに縛られず最強になった～,無能と蔑まれた俺が世界最強になるまでの軌跡,世界中に突如としてダンジョンが出現してから約２０年。\n 人類はレベルとステータス、そしてスキルという超常的な力を手に入れた。\n\n 一獲千金を夢見てダンジョン攻略に挑む冒険者の一人、天音 凛（あまね りん）。\n 彼は【ダンジョン内転移】という冒険者でも特別珍しいユニークスキルを持つが、あまりの使い勝手の悪さから無能だと蔑まれてきた。\n\n しかし、そんな彼に転機が訪れる。\n ダンジョン攻略を始めてからちょうど一年――レベルアップを機に、【ダンジョン内転移】が進化を遂げる。\n その結果、凛はダンジョンのルールに縛られない存在となった。\n\n 世界で唯一、特別な存在になった...
14647,異世界居酒屋「のぶ」,路地裏居酒屋「のぶ」,古都の路地裏に一風変わった店がある。\n居酒屋「のぶ」\nこれは、一軒の居酒屋を巡る、小さな物語である。
419111,本好きの下剋上 ～司書になるためには手段を選んでいられません～,図書館司書の異世界生活,本が好きで、司書資格を取り、大学図書館への就職が決まっていたのに、大学卒業直後に死んでしまった麗乃。転生したのは、識字率が低くて本が少ない世界の兵士の娘。いくら読みたくても周りに本なんてあるはずない。本がないならどうする？ 作ってしまえばいいじゃない。目指すは図書館司書！ 本に囲まれて生きるため、本を作るところから始めよう。※最初の主人公の性格が最悪です。ある程度成長するまで、気分悪くなる恐れがあります。（R15は念のため）


In [28]:
narou_all.sort_values("all_point",ascending=False)

,title,ncode,userid,writer,story,biggenre,genre,gensaku,keyword,general_firstup,general_lastup,novel_type,end,general_all_no,length,time,isstop,isr15,isbl,isgl,iszankoku,istensei,istenni,pc_or_k,global_point,daily_point,weekly_point,monthly_point,quarter_point,yearly_point,fav_novel_cnt,impression_cnt,review_cnt,all_point,all_hyoka_cnt,sasie_cnt,kaiwaritu,novelupdated_at,updated_at,weekly_unique,log_global_point
16520,ヘルモード ～やり込み好きのゲーマーは廃設定の異世界で無双する～,N3669FW,1015944,ハム男,山田健一は３５歳のサラリーマンだ。やり込み好きで普段からゲームに熱中していたが、昨今のヌルゲー仕様の時代の流れに嘆いた。\nそんな中、『やり込み好きのあなたへ』という謳い文句のサイトに惹かれ、ヘルモードの異世界に転生をすることになる。\n\nこの物語は召喚士アレンによる、なろう小説王道ファンタジーである。\n\nアレンは農奴に転生したため、この世界について何も知らない。\n１０年前、２０年前ゲームショップで手に取ったゲームソフトのように、\nどんなゲームかも分からない状態から、アレンの物語は始まっていくのであった。\n\n書籍化について\n出版社：アース・スターノベル様 イラスト：藻様...,2,201,NaN,R15 残酷な描写あり 異世界転生 身分差 サラリーマン 男主人公 召喚士 農奴 レベル ステータス チート 学園 幼馴染 成りあがり 無双 いずれ主人公最強 勇者と魔王 ESN大賞２,2019-11-17 22:05:56,2021-01-24 20:01:22,1,1,366,1396890,2794,0,1,0,0,1,1,0,2,514655,450,3438,19172,51160,369105,106551,11112,57,301553,31529,0,28,2021-02-05 15:05:02,2021-02-05 19:50:12,254075,13.151254
5185,一億年ボタンを連打した俺は、気付いたら最強になっていた～落第剣士の学院無双～,N1474FH,994271,月島 秀一,【ドラマCD化！ 令和元年、年間1位の人気作！ 書籍版1～5巻が富士見ファンタジア文庫より発売中！】\n 剣術学院の落第剣士に『一億年ボタン』を持ちかけた謎の老人がいた。押せば一億年修業したのと同じ効果が得られる魔法のボタン。落第剣士が疑い半分でボタンを押した次の瞬間、その身に絶大な力が宿るのを感じた。彼は喜んで何度もボタンを押した――押してしまった。\n それは呪われたボタンだ。押した瞬間、彼は一億年という『時の牢獄』に閉じ込められた。最初の百年はこれ幸いと修業に明け暮れた。千年が経ち、自分が何故剣を振っているのかわからなくなった。一万年が経つ頃には、考えることをやめていた。\n 一...,2,201,NaN,R15 残酷な描写あり 主人公最強（予定） 成り上がり 異世界 才能の無い落ちこぼれ 数十億年の鍛錬 理性の化物 精神力の化物 忍耐の化物,2019-02-01 11:37:43,2021-02-04 18:19:07,1,1,420,1052930,2106,0,1,0,0,1,0,0,2,429860,116,790,3694,12666,83679,84928,443,9,260004,27818,0,38,2021-02-04 18:19:07,2021-02-05 19:37:45,44193,12.971217
7636,世界最速のレベルアップ ～無能スキル【ダンジョン内転移】が覚醒した結果、俺だけダンジョンのルールに縛られず最強になった～,N3976GK,484778,八又ナガト,世界中に突如としてダンジョンが出現してから約２０年。\n 人類はレベルとステータス、そしてスキルという超常的な力を手に入れた。\n\n 一獲千金を夢見てダンジョン攻略に挑む冒険者の一人、天音 凛（あまね りん）。\n 彼は【ダンジョン内転移】という冒険者でも特別珍しいユニークスキルを持つが、あまりの使い勝手の悪さから無能だと蔑まれてきた。\n\n しかし、そんな彼に転機が訪れる。\n ダンジョン攻略を始めてからちょうど一年――レベルアップを機に、【ダンジョン内転移】が進化を遂げる。\n その結果、凛はダンジョンのルールに縛られない存在となった。\n\n 世界で唯一、特別な存在になった...,2,202,NaN,R15 残酷な描写あり 現代 ダンジョン 最弱 無能 成長 成り上がり 主人公最強 無双 システム音ちゃん,2020-08-03 22:26:02,2021-02-03 00:00:00,1,1,170,358294,717,0,1,0,0,1,0,0,2,361870,476,3950,29966,66796,359690,54309,3082,16,253252,26703,0,29,2021-02-04 22:25:08,2021-02-05 20:04:05,194815,12.799043
8859,とんでもスキルで異世界放浪メシ,N2710DB,679640,江口 連,❖オーバーラップノベルス様より書籍９巻まで発売中！ 本編コミックは６巻まで、外伝コミック「スイの大冒険」は４巻まで発売中です！❖\n異世界召喚に巻き込まれた俺、向田剛志（むこうだ つよし）。\nステータスの鑑定で俺以外の３人の召喚者は職業欄に『異世界からやって来た勇者』となってるのに、俺だけ『巻き込まれた異世界人』となっていた。\nスキルも聖剣術とか聖槍術とか聖魔法とかいろいろすごいのがそろっている。\nそれなのに俺のは固有スキルで『ネットスーパー』だけだった。\n戦闘スキルじゃない俺はすぐさま役立たず的な扱いに。\nしかも、召喚した国の王様の言い分がいかにも胡散臭い。\nネット小説を...,2,201,NaN,R15 残酷な描写あり 異世界転移 異世界 ネットスーパー 勇者召喚 巻き込まれ ペットが最強 料理 ほのぼの のほほん 旅,2016-01-05 01:24:12,2021-02-01 22:19:18,1,1,554,1837957,3676,0,1,0,0,1,0,1,2,637144,346,2926,13074,33918,174261,195167,22906,135,246810,26281,1,36,2021-02-02 07:49:13,2021-02-05 18:53:39,307346,13.364753
182759,陰の実力者になりたくて！【web版】,N0611EM,298373,逢沢大介,【web版と書籍版は途中から内容が異なります】\n\n※書籍３巻とコミック２巻好評発売中です！\n\n どこにでもいる平凡な少年は、異世界で最高峰の魔剣士だった。\n\n 彼の名はシド。\n\n『陰の実力者』に憧れる転生者である。\n\n 彼は実力を隠して学園に入学し、理想の『陰の実力者』になるため暗躍する。\n\n これは、おバカな夢を真面目に叶えようとする少年の物語。,2,201,NaN,R15 残酷な描写あり 異世界転生 陰の実力者(最強) 陰の組織(巨大化) 勘違い コメディ 中二病 主人公最強 俺ＴＵＥＥＥ 配下ＴＵＥＥＥ,2017-12-31 19:58:20,2019-09-22 22:26:48,1,1,203,462645,926,1,1,0,0,1,1,0,2,487926,88,784,3038,8680,45395,123417,9824,52,241092,25391,0,39,2019-09-23 19:29:38,2021-02-05 19:20:19,44040,13.097921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164201,駆け抜けなよ仙女さん！,N0437FX,1555652,石彩,自由気ままの仙女が一国の大事に巻き込まれる？,3,307,NaN,オリジナル戦記 伝奇,2019-12-03 00:20:49,2019-12-03 00:20:49,1,1,1,1145,3,1,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,62,2019-12-04 12:40:41,2020-02-04 05:57:37,0,0.000000
461355,暗闇の舞台,N2438DM,880789,漆黒騎士,一人の悪魔が、生まれる話。,3,305,NaN,R15 残酷な描写あり スプラッタ 悪魔 儀式 転生 サーカス,2016-08-20 23:50

## 青春文庫の作品のタイトルを予測

In [29]:
import re

seisyunbunko.csv　は青春文庫のtitleとtextをコピペで貼り付けたcsvです。

In [30]:
text_df = pd.read_csv("../data/seisyunbunko.csv")
text_df

,title,text
0,走れメロス,メロスは激怒した。必ず、かの邪智暴虐じゃちぼうぎゃくの王を除かなければならぬと決意した。メロスには政治がわからぬ。メロスは、村の牧人である。笛を吹き、羊と遊んで暮して来た。けれども邪悪に対しては、人一倍に敏感であった。きょう未明メロスは村を出発し、野を越え山越え、十里はなれた此このシラクスの市にやって来た。メロスには父も、母も無い。女房も無い。十六の、内気な妹と二人暮しだ。この妹は、村の或る律気な一牧人を、近々、花婿はなむことして迎える事になっていた。結婚式も間近かなのである。メロスは、それゆえ、花嫁の衣裳やら祝宴の御馳走やらを買いに、はるばる市にやって来たのだ。先ず、その品々を買い...
1,こころ,私《わたくし》はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これは世間を憚《はば》かる遠慮というよりも、その方が私にとって自然だからである。私はその人の記憶を呼び起すごとに、すぐ「先生」といいたくなる。筆を執《と》っても心持は同じ事である。よそよそしい頭文字《かしらもじ》などはとても使う気にならない。\n 私が先生と知り合いになったのは鎌倉《かまくら》である。その時私はまだ若々しい書生であった。暑中休暇を利用して海水浴に行った友達からぜひ来いという端書《はがき》を受け取ったので、私は多少の金を工面《くめん》して、出掛ける事にした。私は金の工面に...
2,人間失格,私は、その男の写真を三葉、見たことがある。\n 一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹《いとこ》たちかと想像される）庭園の池のほとりに、荒い縞の袴《はかま》をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である。醜く？ けれども、鈍い人たち（つまり、美醜などに関心を持たぬ人たち）は、面白くも何とも無いような顔をして、\n「可愛い坊ちゃんですね」\n といい加減なお世辞を言っても、まんざら空《から》お世辞に聞えないくらいの、謂《い》わ...
3,吾輩は猫である,吾輩《わがはい》は猫である。名前はまだ無い。\n どこで生れたかとんと見当《けんとう》がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番｜獰悪《どうあく》な種族であったそうだ。この書生というのは時々我々を捕《つかま》えて煮《に》て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌《てのひら》に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間とい...
4,羅生門,ある日の暮方の事である。一人の下人《げにん》が、羅生門《らしょうもん》の下で雨やみを待っていた。\n 広い門の下には、この男のほかに誰もいない。ただ、所々｜丹塗《にぬり》の剥《は》げた、大きな円柱《まるばしら》に、蟋蟀《きりぎりす》が一匹とまっている。羅生門が、朱雀大路《すざくおおじ》にある以上は、この男のほかにも、雨やみをする市女笠《いちめがさ》や揉烏帽子《もみえぼし》が、もう二三人はありそうなものである。それが、この男のほかには誰もいない。\n 何故かと云うと、この二三年、京都には、地震とか辻風《つじかぜ》とか火事とか饑饉とか云う災《わざわい》がつづいて起った。そこで洛中《らくちゅ...


In [31]:
text_df["text"] = text_df["text"].str.replace("《.*?》","")
text_df["pred_title"] = pred_text(text_df["text"])
text_df[["title","pred_title","text"]]

/home/shingo/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


  0%|          | 0/5 [00:00<?, ?it/s]

,title,pred_title,text
0,走れメロス,メロスとセリヌンティウス,メロスは激怒した。必ず、かの邪智暴虐じゃちぼうぎゃくの王を除かなければならぬと決意した。メロスには政治がわからぬ。メロスは、村の牧人である。笛を吹き、羊と遊んで暮して来た。けれども邪悪に対しては、人一倍に敏感であった。きょう未明メロスは村を出発し、野を越え山越え、十里はなれた此このシラクスの市にやって来た。メロスには父も、母も無い。女房も無い。十六の、内気な妹と二人暮しだ。この妹は、村の或る律気な一牧人を、近々、花婿はなむことして迎える事になっていた。結婚式も間近かなのである。メロスは、それゆえ、花嫁の衣裳やら祝宴の御馳走やらを買いに、はるばる市にやって来たのだ。先ず、その品々を買い...
1,こころ,先生,私はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これは世間を憚かる遠慮というよりも、その方が私にとって自然だからである。私はその人の記憶を呼び起すごとに、すぐ「先生」といいたくなる。筆を執っても心持は同じ事である。よそよそしい頭文字などはとても使う気にならない。\n 私が先生と知り合いになったのは鎌倉である。その時私はまだ若々しい書生であった。暑中休暇を利用して海水浴に行った友達からぜひ来いという端書を受け取ったので、私は多少の金を工面して、出掛ける事にした。私は金の工面に二、三日を費やした。ところが私が鎌倉に着いて三日と経たないうちに、私を呼...
2,人間失格,池のほとりの坊ちゃん,私は、その男の写真を三葉、見たことがある。\n 一葉は、その男の、幼年時代、とでも言うべきであろうか、十歳前後かと推定される頃の写真であって、その子供が大勢の女のひとに取りかこまれ、（それは、その子供の姉たち、妹たち、それから、従姉妹たちかと想像される）庭園の池のほとりに、荒い縞の袴をはいて立ち、首を三十度ほど左に傾け、醜く笑っている写真である。醜く？ けれども、鈍い人たち（つまり、美醜などに関心を持たぬ人たち）は、面白くも何とも無いような顔をして、\n「可愛い坊ちゃんですね」\n といい加減なお世辞を言っても、まんざら空お世辞に聞えないくらいの、謂わば通俗の「可愛らしさ」みたいな影も...
3,吾輩は猫である,吾輩は猫である,吾輩は猫である。名前はまだ無い。\n どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番｜獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残って...
4,羅生門,羅生門の狐狸,ある日の暮方の事である。一人の下人が、羅生門の下で雨やみを待っていた。\n 広い門の下には、この男のほかに誰もいない。ただ、所々｜丹塗の剥げた、大きな円柱に、蟋蟀が一匹とまっている。羅生門が、朱雀大路にある以上は、この男のほかにも、雨やみをする市女笠や揉烏帽子が、もう二三人はありそうなものである。それが、この男のほかには誰もいない。\n 何故かと云うと、この二三年、京都には、地震とか辻風とか火事とか饑饉とか云う災がつづいて起った。そこで洛中のさびれ方は一通りではない。旧記によると、仏像や仏具を打砕いて、その丹がついたり、金銀の箔がついたりした木を、路ばたにつみ重ねて、薪の料に売っていた...
